In [2]:
#This notebook is implementing a tournament with spatial topology for the axelrod-python library. 
#Here will we only run small tournaments with deterministic strategies and will intersect the result using
#the simple tournament class for the axelrod library

In [3]:
#import the axelrod library
import axelrod as axl

In [4]:
#this is a class that generate matches with spatial structure
class SpatialMatches(axl.RoundRobinMatches):
    """A match generator for spatial tournaments"""
    def __init__(self, players, turns, game, repetitions, edges):
        self.edges = edges
        super(axl.RoundRobinMatches, self).__init__(players, turns, game, repetitions)


    def build_match_chunks(self):
        for edge in self.edges:
            match_params = self.build_single_match_params()
            index_pair = edge
            yield (index_pair, match_params, self.repetitions)
            
    def __len__(self):
        return  len(self.edges)

In [52]:
# and this is a tournament class that generates a tournament with spatial structure 
class SpatialTournament(axl.Tournament):
    """
    A tournament in which the players are allocated in a graph as nodes
    and they players only play the others that are connected to with an edge.
    """

    def __init__(self, players, match_generator=SpatialMatches,
                 name='axelrod', game=None, turns=200, repetitions=10,
                 noise=0,
                 with_morality=True):
        """
        Parameters
        ----------
        players : list
            A list of axelrod.Player objects
        match_generator : class
            A class that must be descended from axelrod.MatchGenerator
        name : string
            A name for the tournament
        game : axelrod.Game
            The game object used to score the tournament
        edges : dictionary
            A dictionary containing the existing edges
        repetitions : integer
            The number of times the round robin should be repeated
        processes : integer
            The number of processes to be used for parallel processing
        noise : float
            The probability that a player's intended action should be flipped
        with_morality : boolean
            Whether morality metrics should be calculated
        """
        super(SpatialTournament, self).__init__(
            players, name=name, game=game, turns=turns,
            repetitions=repetitions, noise=noise, with_morality=with_morality)

        self.edges = edges
        self.match_generator = SpatialMatches(
            players, turns, self.game, repetitions, edges)
        
    def _write_interactions(self, results):
        """Write the interactions to csv."""
        for index_pair, interactions in results.items():
            for interaction in interactions:
                row = list(index_pair)
                row.append(str(self.players[index_pair[0]]))
                row.append(str(self.players[index_pair[1]]))
                history1 = "".join([i[0] for i in interaction])
                history2 = "".join([i[1] for i in interaction])
                row.append(history1)
                row.append(history2)
                self.writer.writerow(row)
                print(row) # take a look at what row looks like 
                self.num_interactions += 1

In [42]:
# create some players 
players = (axl.Cooperator(), axl.Defector(), 
           axl.TitForTat(), axl.Alternator())

In [ ]:
# run a simple tournament that everybody plays with everybody
tournament = axl.Tournament(players, turns=2, repetitions=2)
results = tournament.play()

In [ ]:
#visualise the iteractions so we can compare to the spatial ones
for index_pair, interaction in results.interactions.items():
     player1 = tournament.players[index_pair[0]]
     player2 = tournament.players[index_pair[1]]
     print('%s vs %s: %s' % (player1, player2, interaction)) 

In [64]:
edges = [(1,2)]

In [65]:
sp=SpatialMatches(players, turns=2, game = None , repetitions=2, edges=edges)

In [66]:
list(sp.build_match_chunks())

[((1, 2), (2, None, None, 0), 2)]

In [67]:
spatial_tournament = SpatialTournament(players, match_generator=sp ,turns=2, repetitions=2)

In [68]:
spatial_results= spatial_tournament.play() 
# here we are gettig a error in the result_set.py 

[1, 2, 'Defector', 'Tit For Tat', 'DD', 'CD']
[1, 2, 'Defector', 'Tit For Tat', 'DD', 'CD']


KeyError: 0

In [73]:
# Fix various sets of edges, so by using a for loop we can run all those different combinatios
# and take a look at the printed results
edges = [[(0, 1)], [(1, 2)], [(1, 3)], [(2, 3)], # with one interaction
         [(1, 2), (2, 3)], [(0, 1), (1, 3)], [(0, 2), (0, 3)], # with two interactions
         [(0, 1), (1, 2), (2, 3)]] #and with three  

In [84]:
for i in range(len(edges)):
      sp=SpatialMatches(players, turns=2, game = None , repetitions=2, edges=edges[i])
      spatial_tournament = SpatialTournament(players, match_generator=sp ,turns=2, repetitions=2)
    
      spatial_results.ranked_names # spatial result is not created due to above error

NameError: name 'spatial_results' is not defined